In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
import pandas as pd
from datetime import datetime
from IPython.core.display import HTML
HTML("<style>.container { width:100%; }</style>")

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

from dse_do_utils import DataManager, OptimizationEngine,ScenarioManager
from docplex.mp.model import Model

# Optimization Model Informations

In [3]:
MODEL_NAME = 'Unit_Commitment'
SCENARIO_NAME = 'Base_Scenario' 
sm = ScenarioManager(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, project = project)

mdl = Model(MODEL_NAME)

In [4]:
InputTables, outputTables = sm.load_data_from_scenario()
InputTables.keys()

dict_keys(['Units', 'Demand'])

In [5]:
Units = InputTables['Units']

display(Units.head(2))

Demand = InputTables['Demand']

display(Demand.head(2))

,index,energy,initial,min_gen,max_gen,operating_max_gen,min_uptime,min_downtime,ramp_up,ramp_down,start_cost,fixed_cost,variable_cost,Unit,co2_cost
0,0,coal,400,100.0,425,400,15,9,212.0,183.0,5000,208.61,22.536,coal1,30
1,1,coal,350,140.0,365,350,15,8,150.0,198.0,4550,117.37,31.985,coal2,30


,demand,Date,Date_Sequence
0,1259.0,2022-04-22,day1
1,1439.0,2022-04-23,day2


# Defining of Functions

Define a set of functions, which will be used later in the code.

In [7]:
def continuous_var_series(df, mdl,**kargs):
    return pd.Series(mdl.continuous_var_list(df.index, **kargs), index = df.index)

def binary_var_series(df, mdl,**kargs):
    return pd.Series(mdl.binary_var_list(df.index, **kargs), index = df.index)

def integer_var_series(df, mdl,**kargs):
    '''Create a Series of integer dvar for each row in the DF. Most effective method. Best practice.
    Result can be assigned to a column of the df.
    Usage:
        df['xDVar'] = mdl.integer_var_series(df, name = 'xDVar')
    Args:
        mdl: CPLEX Model
        df: DataFrame
        **kargs: arguments passed to mdl.integer_var_list method. E.g. 'name'
        
    :returns: pandas.Series with integer dvars, index matches index of df
    '''
    #We are re-using the index from the DF index:
    return pd.Series(mdl.integer_var_list(df.index, **kargs), index = df.index)


class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)

# Define decision variables

We define the decision variables based on each day and unit. First we create a table of all possible permutations

In [11]:
Unit_Date = pd.merge(Units[["Unit"]],Demand["Date_Sequence"], how = "cross" )

Unit_Date = Unit_Date.set_index(["Unit","Date_Sequence"], verify_integrity = True)

Unit_Date.sample(5)

,
Unit,Date_Sequence
gas1,day148
gas2,day181
gas3,day14
diesel2,day68
gas1,day129


The $x_{InUse} (i,t)$ is true if the unit $i$ is in production at period $t$

In [17]:
Unit_Date["x_InUse"] = binary_var_series(Unit_Date, mdl,name = 'x_InUse')

The $x_{TurnOn} (i,t)$  true if unit $i$ is turned on at period $t$

In [18]:
Unit_Date["x_TurnOn"] = binary_var_series(Unit_Date, mdl,name = 'x_TurnOn')

$x_{TurnOff} (i,t)$ is true if unit $i$ is switched off at period $t$

In [19]:
Unit_Date["x_TurnOff"] = binary_var_series(Unit_Date, mdl,name = 'x_TurnOff')                #continuous_var_series

 production of energy for unit u at period t

In [20]:
Unit_Date["x_Production"] = continuous_var_series(Unit_Date, mdl,name = 'x_Production')

In [21]:
Unit_Date = Unit_Date.reset_index(drop = False)

In [22]:
Unit_Date.head()

,index,Unit,Date_Sequence,x_TurnOff,x_Production,x_InUse,x_TurnOn
0,0,coal1,day1,x_TurnOff_0,x_Production_0,x_InUse_0,x_TurnOn_0
1,1,coal1,day2,x_TurnOff_1,x_Production_1,x_InUse_1,x_TurnOn_1
2,2,coal1,day3,x_TurnOff_2,x_Production_2,x_InUse_2,x_TurnOn_2
3,3,coal1,day4,x_TurnOff_3,x_Production_3,x_InUse_3,x_TurnOn_3
4,4,coal1,day5,x_TurnOff_4,x_Production_4,x_InUse_4,x_TurnOn_4


In [23]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 11520
   - binary=7680, integer=0, continuous=3840
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Constraints

# 2- Linking in-use status to production
Whenever the unit is in use, the production must be within the minimum and maximum generation.

In [24]:
Production_range = pd.merge ( Unit_Date[["Unit","x_InUse","x_Production"]] , Units[["Unit","min_gen","max_gen"]], on = "Unit")

Production_range.head()

,Unit,x_InUse,x_Production,min_gen,max_gen
0,coal1,x_InUse_0,x_Production_0,100.0,425
1,coal1,x_InUse_1,x_Production_1,100.0,425
2,coal1,x_InUse_2,x_Production_2,100.0,425
3,coal1,x_InUse_3,x_Production_3,100.0,425
4,coal1,x_InUse_4,x_Production_4,100.0,425


When in use, the production level is constrained to be between min and max generation.

In [25]:
for row in Production_range.itertuples():
    mdl.add_constraint ( row.x_Production <= row.x_InUse * row.max_gen )
    mdl.add_constraint ( row.x_Production >= row.x_InUse * row.min_gen )

In [26]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 11520
   - binary=7680, integer=0, continuous=3840
 - number of constraints: 3840
   - linear=3840
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# 3 -Initial state
If initial production is nonzero, then period #1 is not a turn_on
else turn_on equals in_use
Dual logic is implemented for turn_off

In [27]:
initial_state  = Units [["Unit","initial"]]
day_1 = Unit_Date[Unit_Date["Date_Sequence"]=="day1"]

initial_state = pd.merge (initial_state, day_1, on = "Unit")
initial_state

,Unit,initial,index,Date_Sequence,x_TurnOff,x_Production,x_InUse,x_TurnOn
0,coal1,400,0,day1,x_TurnOff_0,x_Production_0,x_InUse_0,x_TurnOn_0
1,coal2,350,192,day1,x_TurnOff_192,x_Production_192,x_InUse_192,x_TurnOn_192
2,gas1,205,384,day1,x_TurnOff_384,x_Production_384,x_InUse_384,x_TurnOn_384
3,gas2,52,576,day1,x_TurnOff_576,x_Production_576,x_InUse_576,x_TurnOn_576
4,gas3,155,768,day1,x_TurnOff_768,x_Production_768,x_InUse_768,x_TurnOn_768
5,gas4,150,960,day1,x_TurnOff_960,x_Production_960,x_InUse_960,x_TurnOn_960
6,diesel1,78,1152,day1,x_TurnOff_1152,x_Production_1152,x_InUse_1152,x_TurnOn_1152
7,diesel2,76,1344,day1,x_TurnOff_1344,x_Production_1344,x_InUse_1344,x_TurnOn_1344
8,diesel3,0,1536,day1,x_TurnOff_1536,x_Production_1536,x_InUse_1536,x_TurnOn_1536
9,diesel4,0,1728,day1,x_TurnOff_1728,x_Production_1728,x_InUse_1728,x_TurnOn_1728


In [28]:
for row in initial_state.itertuples():
    if row.initial > 0 :   
            
            mdl.add_constraint  ( row.x_TurnOn == 0)                   #if u is already running, not starting up
            
            mdl.add_constraint  ( row.x_TurnOff +  row.x_InUse == 1)   # turnoff iff not in use
            
            
    else: 
        
           mdl.add_constraint ( row.x_TurnOn  == row.x_InUse )        # turn on at 1 iff in use at 1
            
           mdl.add_constraint(row.x_TurnOff ==0)                      # already off, not switched off at t==1

In [29]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 11520
   - binary=7680, integer=0, continuous=3840
 - number of constraints: 3860
   - linear=3860
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# 4 Ramp-up / ramp-down constraint

ramp up and down constraints for day 1
    
Variations of the production level over time in a unit is constrained by a ramp-up / ramp-down process.

We use the  pandas groupby operation to collect all decision variables for each unit in separate series. Then, we iterate over units to post constraints enforcing the ramp-up / ramp-down process by setting upper bounds on the variation of the production level for consecutive periods.

In [30]:
for unit in Units["Unit"].values:
    a = Unit_Date [ (Unit_Date["Unit"]==unit) & (Unit_Date["Date_Sequence"]=="day1")]
    b = Units [ Units["Unit"]==unit][["Unit","initial","ramp_up","ramp_down"]]
    r = pd.merge (a , b,  on ="Unit")
    
    mdl.add_constraint ( r.x_Production[0] - r.initial[0] <= r.ramp_up[0] )
    
    mdl.add_constraint( r.initial[0] - r.x_Production[0] <= r.ramp_down[0])
    

    # ramp up and down constraints for remaining days

In [31]:
Ramp_Table = pd.DataFrame()

for unit in Units["Unit"].values:
    a = Unit_Date[Unit_Date["Unit"]==unit][["Unit","Date_Sequence","x_Production"]]
    a.rename(columns = {"x_Production":"Current_Day_Production"}, inplace = True)   
    a["Next_Day_Production"] = a["Current_Day_Production"].shift(-1)       # shift 1 day to calculate next day production
    a = a.iloc[0:-1]                                                       # get rid of last row, which has a NaN for next day production
    
    Ramp_Table = pd.concat([Ramp_Table,a])

Ramp_Table = pd.merge(Ramp_Table,Units[["Unit",'ramp_up','ramp_down']], on = "Unit") 

Ramp_Table.sample(7)

,Unit,Date_Sequence,Current_Day_Production,Next_Day_Production,ramp_up,ramp_down
331,coal2,day141,x_Production_332,x_Production_333,150.0,198.0
670,gas2,day98,x_Production_673,x_Production_674,94.8,101.7
1057,gas4,day103,x_Production_1062,x_Production_1063,50.0,60.0
1470,diesel2,day134,x_Production_1477,x_Production_1478,60.0,45.0
1268,diesel1,day123,x_Production_1274,x_Production_1275,40.0,24.0
1558,diesel3,day31,x_Production_1566,x_Production_1567,20.0,20.0
1295,diesel1,day150,x_Production_1301,x_Production_1302,40.0,24.0


In [32]:
for row in Ramp_Table.itertuples():
    mdl.add_constraint(row.Next_Day_Production - row.Current_Day_Production <= row.ramp_up)
    
    mdl.add_constraint(row.Current_Day_Production - row.Next_Day_Production <= row.ramp_down)

In [33]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 11520
   - binary=7680, integer=0, continuous=3840
 - number of constraints: 7700
   - linear=7700
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# 5 Turn on / turn off
The following constraints determine when a unit is turned on or off.

We use the same pandas groupby operation as in the previous constraint to iterate over the sequence of decision variables for each unit.

In [34]:
Status = pd.DataFrame([])

for unit in Units["Unit"].values:
    a = Unit_Date[Unit_Date["Unit"]==unit][["Unit","x_InUse","x_TurnOn","x_TurnOff"]]
    
    a.rename(columns = {"x_InUse":"Current_Day_InUse"}, inplace = True)
    a.rename(columns = {"x_TurnOn":"Current_Day_TurnOn"}, inplace = True)
    a.rename(columns = {"x_TurnOff":"Current_Day_TurnOff"}, inplace = True)
    
    a["Next_Day_InUse"] = a["Current_Day_InUse"].shift(-1)       # shift 1 day to calculate next day InUse
    a["Next_Day_TurnOn"] = a["Current_Day_TurnOn"].shift(-1)       # shift 1 day to calculate next day TurnOn
    a["Next_Day_TurnOff"] = a["Current_Day_TurnOff"].shift(-1)       # shift 1 day to calculate next day TurnOff
    
    a = a.iloc[0:-1]                                                # get rid of last row, which has a NaN
    
    Status = pd.concat([Status,a])
        
display(Status.sample(10))

,Unit,Current_Day_InUse,Current_Day_TurnOn,Current_Day_TurnOff,Next_Day_InUse,Next_Day_TurnOn,Next_Day_TurnOff
1571,diesel3,x_InUse_1571,x_TurnOn_1571,x_TurnOff_1571,x_InUse_1572,x_TurnOn_1572,x_TurnOff_1572
1150,gas4,x_InUse_1150,x_TurnOn_1150,x_TurnOff_1150,x_InUse_1151,x_TurnOn_1151,x_TurnOff_1151
1513,diesel2,x_InUse_1513,x_TurnOn_1513,x_TurnOff_1513,x_InUse_1514,x_TurnOn_1514,x_TurnOff_1514
1457,diesel2,x_InUse_1457,x_TurnOn_1457,x_TurnOff_1457,x_InUse_1458,x_TurnOn_1458,x_TurnOff_1458
1310,diesel1,x_InUse_1310,x_TurnOn_1310,x_TurnOff_1310,x_InUse_1311,x_TurnOn_1311,x_TurnOff_1311
602,gas2,x_InUse_602,x_TurnOn_602,x_TurnOff_602,x_InUse_603,x_TurnOn_603,x_TurnOff_603
1518,diesel2,x_InUse_1518,x_TurnOn_1518,x_TurnOff_1518,x_InUse_1519,x_TurnOn_1519,x_TurnOff_1519
631,gas2,x_InUse_631,x_TurnOn_631,x_TurnOff_631,x_InUse_632,x_TurnOn_632,x_TurnOff_632
881,gas3,x_InUse_881,x_TurnOn_881,x_TurnOff_881,x_InUse_882,x_TurnOn_882,x_TurnOff_882
784,gas3,x_InUse_784,x_TurnOn_784,x_TurnOff_784,x_InUse_785,x_TurnOn_785,x_TurnOff_785


if unit is off at time t and on at time t+1, then it was turned on at time t+1

if unit is on at time t and off at time t+1, then it was turned off at time t+1

In [35]:
for row in Status.itertuples():
    
    #print(row.Next_Day_InUse - row.Current_Day_InUse <= row.Next_Day_TurnOn)
    mdl.add_constraint(row.Next_Day_InUse - row.Current_Day_InUse <= row.Next_Day_TurnOn)
    
    #print(row.Current_Day_InUse - row.Next_Day_InUse + row.Next_Day_TurnOn == row.Next_Day_TurnOff)
    mdl.add_constraint(row.Current_Day_InUse - row.Next_Day_InUse + row.Next_Day_TurnOn == row.Next_Day_TurnOff)

In [36]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 11520
   - binary=7680, integer=0, continuous=3840
 - number of constraints: 11520
   - linear=11520
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# 6 Minimum uptime and downtime
When a unit is turned on, it cannot be turned off before a minimum uptime. Conversely, when a unit is turned off, it cannot be turned on again before a minimum downtime.

Again, let's use the same pandas groupby operation to implement this constraint for each unit.

In [37]:
T = len(Demand)

for unit in Units["Unit"].values:
    a= Units[Units["Unit"]==unit][["Unit","min_uptime","min_downtime"]]
    
    min_uptime = a.min_uptime.values[0] 
    min_downtime = a.min_downtime.values[0] 
    
    for t in range(min_uptime,T):
        mdl.add_constraint(mdl.sum(  Unit_Date[(t - min_uptime) + 1   :   t + 1 ][["x_TurnOn"]].values) <= Unit_Date["x_InUse"][t] ) 
        
        
    for t in range(min_downtime, T):   
        mdl.add_constraint(mdl.sum( Unit_Date[(t - min_downtime) + 1:t + 1]["x_TurnOff"].values)  <= 1 - Unit_Date["x_InUse"][t]  ) 

In [38]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 11520
   - binary=7680, integer=0, continuous=3840
 - number of constraints: 15263
   - linear=15263
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# 1- Demand

In [39]:
Total_day_Production = Unit_Date[["Date_Sequence","x_Production"]].groupby("Date_Sequence").agg(CplexSum(mdl)).rename(columns={"x_Production":"Total_day_Production"})
Total_day_Production = pd.merge(Total_day_Production, Demand, on = "Date_Sequence")

Total_day_Production.head(2)

,Date_Sequence,Total_day_Production,demand,Date
0,day1,x_Production_0+x_Production_192+x_Production_384+x_Production_576+x_Production_768+x_Production_960+x_Production_1152+x_Production_1344+x_Production_1536+x_Production_1728,1259.0,2022-04-22
1,day10,x_Production_9+x_Production_201+x_Production_393+x_Production_585+x_Production_777+x_Production_969+x_Production_1161+x_Production_1353+x_Production_1545+x_Production_1737,1158.0,2022-05-01


In [40]:
for row in Total_day_Production.itertuples():
    
    mdl.add_constraint(row.Total_day_Production >= row.demand)

In [41]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 11520
   - binary=7680, integer=0, continuous=3840
 - number of constraints: 15455
   - linear=15455
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [91]:
Units.head(1)

,index,energy,initial,min_gen,max_gen,operating_max_gen,min_uptime,min_downtime,ramp_up,ramp_down,start_cost,fixed_cost,variable_cost,Unit,co2_cost
0,0,coal,400,100.0,425,400,15,9,212.0,183.0,5000,208.61,22.536,coal1,30


In [92]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=3840, integer=0, continuous=3840
 - number of constraints: 15455
   - linear=15455
 - parameters: defaults
 - objective: none
 - problem type is: MILP


demand constraint

Total production level must be equal or higher than demand on any period.

This time, the pandas operation groupby is performed on "periods" since we have to iterate over the list of all units for each period.

In [93]:
Unit_Date.head(1)

,Unit,Date_Sequence,x_InUse,x_TurnOn,x_TurnOff,x_Production
0,coal1,day1,x_InUse_coal1_day1,x_TurnOn_coal1_day1,x_TurnOff_coal1_day1,x_Production_coal1_day1


In [94]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=3840, integer=0, continuous=3840
 - number of constraints: 15455
   - linear=15455
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# KPIs

In [42]:
cost = pd.merge ( Unit_Date, Units[["Unit","fixed_cost","variable_cost","start_cost","co2_cost"]], on = "Unit")
cost.head()

,index,Unit,Date_Sequence,x_TurnOff,x_Production,x_InUse,x_TurnOn,fixed_cost,variable_cost,start_cost,co2_cost
0,0,coal1,day1,x_TurnOff_0,x_Production_0,x_InUse_0,x_TurnOn_0,208.61,22.536,5000,30
1,1,coal1,day2,x_TurnOff_1,x_Production_1,x_InUse_1,x_TurnOn_1,208.61,22.536,5000,30
2,2,coal1,day3,x_TurnOff_2,x_Production_2,x_InUse_2,x_TurnOn_2,208.61,22.536,5000,30
3,3,coal1,day4,x_TurnOff_3,x_Production_3,x_InUse_3,x_TurnOn_3,208.61,22.536,5000,30
4,4,coal1,day5,x_TurnOff_4,x_Production_4,x_InUse_4,x_TurnOn_4,208.61,22.536,5000,30


In [43]:
total_fixed_cost = mdl.sum(cost.x_InUse * cost.fixed_cost)
total_variable_cost = mdl.sum(cost.x_Production * cost.variable_cost)
total_startup_cost = mdl.sum(cost.x_TurnOn * cost.start_cost)
total_co2_cost = mdl.sum(cost.x_Production * cost.co2_cost)

total_economic_cost = total_fixed_cost + total_variable_cost + total_startup_cost

total_nb_used = mdl.sum(Unit_Date.x_InUse)
total_nb_starts = mdl.sum(Unit_Date.x_TurnOn)


# store expression kpis to retrieve them later.
mdl.add_kpi(total_fixed_cost   , "Total Fixed Cost")
mdl.add_kpi(total_variable_cost, "Total Variable Cost")
mdl.add_kpi(total_startup_cost , "Total Startup Cost")
mdl.add_kpi(total_economic_cost, "Total Economic Cost")
mdl.add_kpi(total_co2_cost     , "Total CO2 Cost")
mdl.add_kpi(total_nb_used, "Total #used")
mdl.add_kpi(total_nb_starts, "Total #starts")

# minimize sum of all costs
mdl.minimize(total_fixed_cost + total_variable_cost + total_startup_cost + total_co2_cost)

Solve

In [44]:
mdl.solve()

def extract_solution(df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [dvar.solution_value for dvar in df[xDVarName]]
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df


In [45]:
mdl.report()

* model Unit_Commitment solved with objective = 14209787.418
*  KPI: Total Fixed Cost    = 157934.630
*  KPI: Total Variable Cost = 8831892.288
*  KPI: Total Startup Cost  = 13439.000
*  KPI: Total Economic Cost = 9003265.918
*  KPI: Total CO2 Cost      = 5206521.500
*  KPI: Total #used         = 1318.000
*  KPI: Total #starts       = 12.000


# Create output tables

In [46]:
outputs = {}

In [47]:
plan = extract_solution(Unit_Date,extract_dvar_names= ['x_TurnOn','x_TurnOff','x_Production','x_InUse'] , drop=True)
plan.round(2).head()

,index,Unit,Date_Sequence,x_TurnOn_Solution,x_TurnOff_Solution,x_Production_Solution,x_InUse_Solution
0,0,coal1,day1,0.0,0.0,425.0,1.0
1,1,coal1,day2,0.0,0.0,425.0,1.0
2,2,coal1,day3,0.0,0.0,425.0,1.0
3,3,coal1,day4,0.0,0.0,425.0,1.0
4,4,coal1,day5,0.0,0.0,425.0,1.0


In [48]:
Power_portions = plan[["Unit","Date_Sequence","x_Production_Solution"]].groupby(["Date_Sequence","Unit"]).sum().round().reset_index(drop = False)
Power_portions = pd.merge(Power_portions, Demand[["Date","Date_Sequence"]], on = "Date_Sequence")
del Power_portions["Date_Sequence"]
Power_portions.head()

,Unit,x_Production_Solution,Date
0,coal1,425.0,2022-04-22
1,coal2,215.0,2022-04-22
2,diesel1,90.0,2022-04-22
3,diesel2,87.0,2022-04-22
4,diesel3,0.0,2022-04-22


In [49]:
plant_percentage = plan[["Unit","Date_Sequence","x_Production_Solution"]].groupby(["Unit"]).sum().round().reset_index(drop = False)

plant_percentage["percentage"] = plant_percentage["x_Production_Solution"]/plant_percentage["x_Production_Solution"].sum()

In [50]:
#plan[["Date_Sequence","x_InUse_Solution"]].groupby("Date_Sequence").agg(sum).plot()

In [51]:
outputs['plan'] = plan
outputs['Power_portions'] = Power_portions
outputs['plant_percentage'] = plant_percentage

In [52]:
sm.outputs = outputs
sm.update_solve_output_into_scenario(mdl, outputs)

In [58]:
sm.write_data_to_csv()

Failed to lookup project.
Status 401 for GET /v2/projects/a3018ff3-48f5-45b0-a97a-a331ec53d068
{"code":401,"error":"Unauthorized","reason":"Bearer token is either missing or invalid: undefined.","message":"Access denied"}


Writing /home/wsuser/work/Units.csv


RuntimeError: Failed to lookup project, status 401.

In [57]:
?sm.write_data_to_csv